# 0. Preamble

## 0.1. Connect to the Google shared folder "Capstone"

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/Capstone/'

/content/drive/My Drive/Capstone


In [3]:
ls

 chroma_database/         final_eval_details.gsheet
 chroma_test/            'GitHub related'/
 credentials/             Notebooks/
 datasets/                {persist_base_directory}vector_store_rcts_1500_small/
 Deliverables/            response_eval_accuracy.csv
 eval/                    response_eval_details.csv
 final_eval_details.csv


## 0.2. Install Dependencies

please install langchain required libraries: reference documentation  https://python.langchain.com/docs/how_to/installation/
<br> you will need your Open AI API key  https://platform.openai.com/api-keys


In [4]:
!pip install langchain
!pip install langchain-community
!pip install langchain-experimental
!pip install pypdf
!pip install unstructured
!pip install lxml html5lib
!pip install langchain-openai
!pip install chromadb
!pip install openai
!pip install -U langchain-chroma
!pip install colbert
!pip install groq
!pip install --upgrade langchain openai
!pip install transformers -U



# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.
# tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.

# Dependency Conflict: tensorflow and tensorflow-metadata require a specific version of protobuf (<5 and >=3.20.3), but the version installed (protobuf 5.29.0) does not meet this requirement.
# Successful Installation: Despite the warning, chromadb and its dependencies were installed successfully.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/

In [ ]:
# install if using GPU

!pip install transformers torch accelerate bitsandbytes
!pip install -U bitsandbytes
!pip install --upgrade transformers bitsandbytes accelerate

# !pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


## 0.3. Imports

In [5]:
from dotenv import load_dotenv
import os
import time
import re
import math
import pickle
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gspread
import shutil
from sklearn.preprocessing import normalize

import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig

import openai

from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain.embeddings.base import Embeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

from langchain.schema import SystemMessage, HumanMessage

import groq
from groq import Client, Groq

# from google.colab import userdata

## 0.4. API Keys

<font color=red>delete below cell when share</font>

In [ ]:
# replace with your credentials
env_content = """
GROQ_API_KEY=
OPENAI_API_KEY=
HF_TOKEN=
"""

with open('credentials/claire.env', 'w') as f:   # replace it with your env filename
    f.write(env_content)

In [6]:
load_dotenv('credentials/claire.env', override=True) # replace it with your env filename

groq_api_key = os.getenv('GROQ_API_KEY')
huggingface_key = os.getenv('HF_TOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

<br>

# 1. Loading Vector Stores

In [7]:
embedding_small = OpenAIEmbeddings(model ="text-embedding-3-small")

embedding_ada = OpenAIEmbeddings(model="text-embedding-ada-002")

In [8]:
# FinBERT are tailored to financial or legal documents
class FinBERTEmbeddings(Embeddings):
    def __init__(self, model_name="yiyanghkust/finbert-tone", max_length=512):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.max_length = max_length

    def embed_documents(self, documents):
        # Convert a list of documents to embeddings
        return [self._embed_text(doc) for doc in documents]

    def embed_query(self, query):
        # Convert a single query to an embedding
        return self._embed_text(query)

    def _embed_text(self, text):
        # Tokenize and encode the text
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=self.max_length)
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Use the mean of the token embeddings (can customize based on use case)
            embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        return embeddings

embedding_FinBERT = FinBERTEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# # manual load for testing purpose only
# vector_store_rcts_1500_small = Chroma(
#     persist_directory="chroma_database/vector_store_rcts_1500_small",
#     embedding_function=embedding_small
# )

# vector_store = vector_store_rcts_1500_small

In [13]:
def load_vector_stores(base_directory, embedding_models):
    """
    Loads all vector stores from the given base directory dynamically based on the naming convention.

    :param base_directory: Directory where the vector stores are saved
    :param embedding_models: Dictionary mapping embedding names to embedding objects
    :return: Dictionary of loaded vector stores
    """
    vector_stores = {}

    for root, dirs, files in os.walk(base_directory):
        for dir_name in dirs:
            # Parse directory name to extract chunk type, size, and embedding model
            parts = dir_name.split("_")
            if len(parts) >= 4:
                chunk_type = parts[2]
                chunk_size = parts[3]
                embedding_name = parts[4]

                # Check if the embedding model exists
                if embedding_name in embedding_models:
                    persist_directory = os.path.join(root, dir_name)
                    embedding_model = embedding_models[embedding_name]

                    # Load the vector store
                    vector_store_key = f"vector_store_{chunk_type}_{chunk_size}_{embedding_name}"
                    vector_stores[vector_store_key] = Chroma(
                        persist_directory=persist_directory,
                        embedding_function=embedding_model
                    )

    return vector_stores

# Embedding models
embedding_models = {
    "small": embedding_small,
    "ada": embedding_ada,
    "FinBERT": embedding_FinBERT
}

# Load all vector stores
base_directory = "chroma_database"
all_vector_stores = load_vector_stores(base_directory, embedding_models)

In [14]:
print(f'Successfully loaded {len(all_vector_stores.keys())} vector stores:')
print(all_vector_stores.keys())

Successfully loaded 18 vector stores:
dict_keys(['vector_store_rcts_500_small', 'vector_store_rcts_1000_small', 'vector_store_rcts_1500_small', 'vector_store_semantic_500_small', 'vector_store_semantic_1000_small', 'vector_store_semantic_1500_small', 'vector_store_rcts_500_ada', 'vector_store_rcts_1000_ada', 'vector_store_rcts_1500_ada', 'vector_store_semantic_500_ada', 'vector_store_semantic_1000_ada', 'vector_store_semantic_1500_ada', 'vector_store_rcts_500_FinBERT', 'vector_store_rcts_1000_FinBERT', 'vector_store_rcts_1500_FinBERT', 'vector_store_semantic_500_FinBERT', 'vector_store_semantic_1000_FinBERT', 'vector_store_semantic_1500_FinBERT'])


In [15]:
# initialize default vector store

vector_store = all_vector_stores['vector_store_rcts_1500_small']

<br>

# 2. Chat bot with RAG

## 2.1. Models

### 2.1.1. ChatGPT - via OpenAI API

In [16]:
max_tokens = 4096  # Limit the number of tokens in responses
temperature = 0 # Controls response randomness; The lower the temperature, the more focused the distribution becomes on the most likely tokens

gpt3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=temperature,
    max_tokens=max_tokens,
)


gpt4 = ChatOpenAI(
    model="gpt-4",
    temperature=temperature,
    max_tokens=max_tokens,
)

gpt4o = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=temperature,
    max_tokens=max_tokens,
)

### 2.1.2. LlaMa 3.1. - via GROQ API

In [17]:
groq_client = Groq(api_key=groq_api_key)

In [18]:
# Create a class or function that mimics the interface of ChatOpenAI for LlaMa

class LlamaChat:
    def __init__(self, client, model, temperature=0, max_tokens=4096):
        """
        Initialize the LlamaChat wrapper.

        Parameters:
            client: Groq client instance.
            model (str): Llama model name.
            temperature (float): Sampling temperature.
            max_tokens (int): Maximum token limit for responses.
        """
        self.client = client
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens

    def invoke(self, messages):
        """
        Mimics the ChatOpenAI `invoke` method for Llama, handling both LangChain-style messages
        and single-string prompts.

        Parameters:
            messages (list or str): Either a list of LangChain messages or a single string prompt.

        Returns:
            A response object with a `.content` attribute for compatibility.
        """
        # Handle single-string prompts
        if isinstance(messages, str):
            prompt = messages
        else:
            # Convert LangChain messages to a single prompt
            prompt = "\n".join([msg.content for msg in messages])

        # Call the Llama model via Groq
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )

        # Extract the content of the response using attribute access
        class Response:
            def __init__(self, content):
                self.content = content

        return Response(content=response.choices[0].message.content)


In [19]:
llama3_1_70b = LlamaChat(
    client=groq_client,
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=4096,
)

llama3_1_8b = LlamaChat(
    client=groq_client,
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=4096,
)


## 2.2. System Prompt & RAG LLM Response

In [20]:
system_prompt = """
You are a financial expert. Use the following context from SEC filings, news, or company presentations to answer the question accurately and concisely.
Use metadata to extract dates and return answers from the most recent dates.
If you don't know the answer, say you don't know.
"""

In [21]:
def rag_response(query=None, system_prompt=system_prompt, rag_model=gpt3, vector_store=vector_store, top_k=3):
    """
    Generates an answer using the ChatOpenAI model with context retrieved from a vector store.

    Parameters:
        model (ChatOpenAI): The initialized ChatOpenAI model.
        vector_store (Chroma): The vector store to retrieve relevant context.
        system_prompt (str): The system's instructions to the model.
        query (str): The user query.
        top_k (int): The number of top relevant documents to retrieve from the vector store.

    Returns:
        str: The model's response.
    """
    if query:

      # Step 1: Retrieve relevant context from the vector store
      docs = vector_store.similarity_search(query, k=top_k)
      context = "\n".join([doc.page_content for doc in docs])

      # Step 2: Construct the messages
      messages = [
          SystemMessage(content=system_prompt),
          HumanMessage(content=f"Context:\n{context}\n\nQuestion: {query}")
      ]

      # Step 3: Get the response from the model
      response = rag_model.invoke(messages)

      return response.content

    else: print('please enter a query')

## 2.3. Quick tests

In [22]:
# test one query - use gpt

query = "What is Nvidia's revenue for their last quarter?"

rag_response(query=query)

"NVIDIA's revenue for their last quarter, which ended on October 27, 2024, was $35.1 billion."

In [23]:
# test one query - use llama

query = "What is Nvidia's revenue for their last quarter?"

rag_response(query=query, rag_model=llama3_1_70b)

"NVIDIA's revenue for their last quarter (Q3 Fiscal 2025) is $35.1 billion."

<br>

# 3. Evaluation

## 3.1. Load Eval Question Bank


In [24]:
sheet_path = '/content/drive/My Drive/Capstone/eval/Eval_QA_Bank.xlsx'

eval_qa_bank_ori = pd.read_excel(sheet_path)

In [25]:
eval_qa_bank = eval_qa_bank_ori.copy()

print(eval_qa_bank.shape)
print(eval_qa_bank.dtypes)


(60, 5)
qa_index     int64
query       object
answer      object
source      object
Company     object
dtype: object


In [26]:
eval_qa_bank.head(2)

,qa_index,query,answer,source,Company
0,1,What is the Nvidia's revenue for their last qu...,Nvidia reported its 2025 third-quarter earning...,NVDA_2024-11-20_10-Q.html ; NVIDIA Announces F...,Nvidia
1,2,"For Nvidia, What segments or products contribu...","The Data Center segment, which includes Comput...",NVDA_2024-11-20_10-Q.html,Nvidia


## 3.2. LLM Judge

In [27]:
def llm_judge_withExplanation(query, response, ground_truth, judge_model=gpt4):
    """
    Evaluates the correctness of the chatbot's response using an LLM.

    Args:
        query (str): The input query.
        response (str): The response generated by the chatbot.
        ground_truth (str): The correct answer from the evaluation dataset.
        model (str): The LLM to be used for evaluation (default: "gpt4").

    Returns:
        bool: True if the response is deemed correct by the LLM, False otherwise.
    """
    prompt = (
      f"""Evaluate the accuracy of the chatbot's response based on the following:\n\n
      Query: {query}\n
      Chatbot Response: {response}\n
      Ground Truth: {ground_truth}\n\n
      **Evaluation Criteria:**\n
      1. **Match with Ground Truth:** Respond with 'Correct' if the chatbot's response aligns with the ground truth exactly or provides a reasonable approximation (e.g., minor differences in rounding, formatting, or phrasing that do not change the meaning).\n
      2. **Fiscal vs. Calendar Year Context:** NVIDIA operates on a fiscal year ending in late January. Fiscal year 2025 spans February 2024 to January 2025, with Q3 ending on October 27, 2024. Ensure that the chatbot response properly accounts for this distinction when relevant.\n
      3. **Numerical Equivalence:** Consider numbers equivalent if they fall within standard rounding tolerances (e.g., $35.1 billion ≈ $35,082 million).\n
      4. **Factually Inaccurate or Misaligned Responses:** Respond with 'Incorrect' if the chatbot response:\n
          - Misrepresents the fiscal or calendar year.\n
          - Contains incorrect numerical data outside acceptable rounding tolerances.\n
          - Fails to align with the meaning or context of the ground truth.\n\n
      **Response Format:**\n
      - Reply with either 'True' or 'False'.\n
      - Provide a clear and concise explanation, addressing:\n
          - Any fiscal/calendar year discrepancies.\n
          - Numerical or contextual misalignments.\n
          - Any other reason affecting the accuracy of the response.\n\n
      Example Explanation:\n
      - True: The response accurately reflects NVIDIA’s fiscal year conventions and aligns with the ground truth numerically and contextually.\n
      - False: The response misrepresents the fiscal year as the calendar year, leading to a mismatch with the ground truth.\n\n
      Evaluate carefully and explain your reasoning clearly.
      """
    )


    try:
        response = judge_model.invoke(prompt).content
        return response.strip().lower()
    except Exception as e:
        print(f"Error interacting with the LLM: {e}")
        return "Error"

In [28]:
# test one query on one vector store - use gpt

query = eval_qa_bank.iloc[0]['query']
ground_truth = eval_qa_bank.iloc[0]['answer']
response = rag_response(query)

result = llm_judge_withExplanation(query=query, response=response, ground_truth=ground_truth)
print(f'response judge: {result}\n')
print(f'query: {query}\n')
print(f'ground_truth: {ground_truth}\n')
print(f'response: {response}\n')

response judge: true: the chatbot's response is accurate. it correctly identifies the fiscal year and the quarter end date. the reported revenue of $35.1 billion is also numerically equivalent to the ground truth value of $35,082 million. therefore, the response aligns with the ground truth both numerically and contextually.

query: What is the Nvidia's revenue for their last quarter

ground_truth: Nvidia reported its 2025 third-quarter earnings, announcing revenue of $35,082 millions

response: NVIDIA's revenue for the last quarter, which ended on October 27, 2024, was $35.1 billion.



In [29]:
# test one query on one vector store - use llama

query = eval_qa_bank.iloc[0]['query']
ground_truth = eval_qa_bank.iloc[0]['answer']
response = rag_response(query, rag_model=llama3_1_70b)

result = llm_judge_withExplanation(query=query, response=response, ground_truth=ground_truth, judge_model=llama3_1_8b)
print(f'response judge: {result}\n')
print(f'query: {query}\n')
print(f'ground_truth: {ground_truth}\n')
print(f'response: {response}\n')

response judge: **evaluation result:** true

**explanation:**

1. **match with ground truth:** the chatbot's response, "nvidia's revenue for their last quarter (q3 fiscal 2025) was $35.1 billion," aligns with the ground truth, "nvidia reported its 2025 third-quarter earnings, announcing revenue of $35,082 millions." although the chatbot response uses a slightly different format and rounding, the numerical values are equivalent within standard rounding tolerances.

2. **fiscal vs. calendar year context:** the chatbot response correctly accounts for nvidia's fiscal year conventions by specifying "fiscal 2025" for the quarter in question. this distinction is accurate, as q3 fiscal 2025 corresponds to the period ending in october 2024, which aligns with the ground truth.

3. **numerical equivalence:** as mentioned earlier, the chatbot's response and the ground truth have equivalent numerical values within standard rounding tolerances ($35.1 billion ≈ $35,082 million).

4. **factually inacc

## 3.3. Eval responses

In [30]:
def response_eval(vector_store=vector_store, eval_qa_bank=eval_qa_bank, rag_model=gpt3, judge_model=gpt4, sleep_sec=600):
    """
    Evaluates the chatbot's performance using a given vector store.

    Args:
        vector_store: The vector store to be used for RAG responses.
        eval_qa_bank: DataFrame containing columns 'query' and 'answer' for evaluation.
        rag_model: The model to be used for generating responses (default: gpt3).
        judge_model: The model to be used for judging responses (default: gpt4).

    Returns:
        tuple:
            - float: The accuracy of the chatbot on the given evaluation set.
            - DataFrame: Evaluation results containing 'qa_index', 'response', 'eval',
              'explanation', and 'time_spend' columns.
    """
    eval_results = []
    correct_count = 0
    total_questions = len(eval_qa_bank)

    for idx, row in enumerate(eval_qa_bank.itertuples(index=False), start=1):
        start_time = datetime.now()  # Start time logging

        query = row.query
        ground_truth = row.answer

        # Generate bot response
        response = rag_response(query=query, vector_store=vector_store, rag_model=rag_model)

        # Judge the response
        judge_result = llm_judge_withExplanation(query, response, ground_truth, judge_model)

        # Parse the result and explanation
        if isinstance(judge_result, str):
            explanation = judge_result

            # Use regex to determine correctness
            match = re.search(r'\btrue\b', judge_result.strip(), re.IGNORECASE)
            if match:
                is_correct = match.group(1).lower() == "true"
            else:
                is_correct = False

        else:
            is_correct = False
            explanation = "Error in judging process or invalid format returned."

        # Update correct count
        if is_correct:
            correct_count += 1

        # End time logging
        end_time = datetime.now()
        time_spent = (end_time - start_time).total_seconds()

        # Append results
        eval_results.append({
            "qa_index": row.qa_index,
            "response": response,
            "eval": is_correct,
            "explanation": explanation,
            "time_spend": time_spent
        })

        # Sleep for default 600 seconds after every 10 iterations
        if idx % 10 == 0:
            time.sleep(sleep_sec)

    # Calculate accuracy
    accuracy = correct_count / total_questions

    return accuracy, pd.DataFrame(eval_results)


In [31]:
accuracy, df = response_eval(eval_qa_bank = eval_qa_bank[:1])
print(accuracy)
df

1.0


,qa_index,response,eval,explanation,time_spend
0,1,"NVIDIA's revenue for the last quarter, which e...",True,true: the chatbot's response is accurate. it c...,3.902628


In [32]:
# # for testing purpose

# keys_to_keep = ["vector_store_rcts_1500_small"]
# subset_dict = {key: all_vector_stores[key] for key in keys_to_keep if key in all_vector_stores}

## 3.4. Compare eval results across all vector stores

In [34]:
# use gpt3.5 for RAG model and gpt4 for judge
# replace subset_dict with all_vector_stores

response_eval_accuracy = []
response_eval_results = []

accuracy_file = "eval/response_eval_accuracy_vectorStores.csv"
details_file = "eval/response_eval_details_vectorStores.csv"

for name, vector_store in all_vector_stores.items():
    print(f"Evaluating vector store: {name}")

    # Extract parts of the vector store name
    parts = name.split('_')
    chunking_method = parts[2] if len(parts) > 2 else None
    chunk_size = int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else None
    embedding_model = parts[4] if len(parts) > 4 else None

    # Evaluate the vector store
    accuracy, eval_result = response_eval(vector_store=vector_store, eval_qa_bank=eval_qa_bank, rag_model=gpt3, judge_model=llama3_1_8b, sleep_sec=0) # change model type here

    # Append overall results
    response_eval_accuracy.append({"vector_store": name, "chunking_method": chunking_method, "chunk_size": chunk_size, "embedding_model": embedding_model, "accuracy": accuracy})

    # Add vector store information to the eval_result DataFrame
    eval_result["vector_store"] = name
    eval_result["chunking_method"] = chunking_method
    eval_result["chunk_size"] = chunk_size
    eval_result["embedding_model"] = embedding_model

    # Append all rows of eval_result to the final results
    response_eval_results.append(eval_result)

    # Save periodically
    response_eval_accuracy_df = pd.DataFrame(response_eval_accuracy)
    response_eval_details_df = pd.concat(response_eval_results, ignore_index=True)

    response_eval_accuracy_df.to_csv(accuracy_file, index=False)
    response_eval_details_df.to_csv(details_file, index=False)

In [ ]:
# # use llama 3.1. 70b for RAG model and llama 3.1. 8b for judge

# response_eval_accuracy = []
# response_eval_results = []

# for name, vector_store in all_vector_stores.items():
#     print(f"Evaluating vector store: {name}")

#     # Extract parts of the vector store name
#     parts = name.split('_')
#     chunking_method = parts[2] if len(parts) > 2 else None
#     chunk_size = int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else None
#     embedding_model = parts[4] if len(parts) > 4 else None

#     # Evaluate the vector store
#     accuracy, eval_result = response_eval(vector_store=vector_store, eval_qa_bank=eval_qa_bank, rag_model=llama3_1_70b, judge_model=llama3_1_8b) # change model type here

#     # Append overall results
#     response_eval_accuracy.append({"vector_store": name, "chunking_method": chunking_method, "chunk_size": chunk_size, "embedding_model": embedding_model, "accuracy": accuracy})

#     # Add vector store information to the eval_result DataFrame
#     eval_result["vector_store"] = name
#     eval_result["chunking_method"] = chunking_method
#     eval_result["chunk_size"] = chunk_size
#     eval_result["embedding_model"] = embedding_model

#     # Append all rows of eval_result to the final results
#     response_eval_results.append(eval_result)

#     # Save periodically
#     response_eval_accuracy_df = pd.DataFrame(response_eval_accuracy)
#     response_eval_details_df = pd.concat(response_eval_results, ignore_index=True)

#     response_eval_accuracy_df.to_csv(accuracy_file, index=False)
#     response_eval_details_df.to_csv(details_file, index=False)

## 3.5. Compare eval results across different LLMs

In [35]:
vector_store = all_vector_stores['vector_store_rcts_1500_small']

In [ ]:
llm_models = {'gpt3':gpt3, 'gpt4':gpt4, 'llama3_1_8b':llama3_1_8b, 'llama3_1_70b':llama3_1_70b}

response_eval_accuracy = []
response_eval_details = []

accuracy_file = "eval/response_eval_accuracy_LLM.csv"
details_file = "eval/response_eval_details_LLM.csv"

for llm_name, reg_llm in llm_models.items():
    print(f"Evaluating llm model: {llm_name}")

    # Evaluate the llm model
    if llm_name in ['gpt3','gpt4']:
       sleep_sec = 0
    else:
       sleep_sec = 600

    accuracy, eval_result = response_eval(vector_store=vector_store, eval_qa_bank=eval_qa_bank, rag_model=reg_llm, judge_model=llama3_1_8b, sleep_sec=sleep_sec) # change model type here

    # Append overall results
    response_eval_accuracy.append({"reg_llm": llm_name, "accuracy": accuracy})

    # Add llm model information to the eval_result DataFrame
    eval_result["reg_llm"] = llm_name

    # Append all rows of eval_result to the final results
    response_eval_details.append(eval_result)

    response_eval_accuracy_df = pd.DataFrame(response_eval_accuracy)
    response_eval_details_df = pd.concat(response_eval_details, ignore_index=True)

    response_eval_accuracy_df.to_csv(accuracy_file, index=False)
    response_eval_details_df.to_csv(details_file, index=False)


Evaluating llm model: gpt3
Evaluating llm model: gpt4
Evaluating llm model: llama3_1_8b
Evaluating llm model: llama3_1_70b
